In [35]:
from controller import Imager

Found 2 devices
Devices IDs - ['101935', '192900073']


In [ ]:
imager = Imager()

In [36]:
imager._start_acquisitions()

In [37]:
#imager.update_params(ExposureTime=30000, PixelFormat='Mono8', AcquisitionFrameRate = 10)
imager.get_images(show_images=True)
#imager.get_images(show_images=True)

cam 101935 captured image of format Mono8 at 2021-05-30 19:04:06.390677
cam 192900073 captured image of format BayerRG8 at 2021-05-30 19:04:06.394676


In [38]:
imager.show_images()

In [34]:
imager._stop_acquisitions()
imager.clear_all()